# Laboratorio 01 

## Importar librerías

In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import pickle
from sklearn.metrics import precision_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import json
from sklearn.metrics import confusion_matrix


## Paso 1: Carga y Limpieza de Datos

In [2]:
#agrupar educaciones mayores que 4 (Others)
def agrupar_educaciones(codigo_educacion):
    if codigo_educacion > 4:
        return 4
    return codigo_educacion


def cargar_limpiar_dataset(nombre_archivo):
    datos = pd.read_csv(nombre_archivo)
    
    # - Renombre la columna "default payment next month" a "default".
    datos = datos.rename(columns={"default payment next month" : "default"})

    # Remueva la columna "ID".
    datos.drop(columns=["ID"], inplace = True)

    # Elimine los registros con informacion no disponible.
    datos = datos.dropna()

    # Para la columna EDUCATION, valores > 4 indican niveles superiores de educación, agrupe estos valores en la categoría "others"
    datos["EDUCATION"] = datos["EDUCATION"].apply(agrupar_educaciones)

    return datos

datos_entrenamiento = cargar_limpiar_dataset("../files/input/train_default_of_credit_card_clients.csv")
datos_prueba = cargar_limpiar_dataset("../files/input/test_default_of_credit_card_clients.csv")

print("Datos de Entrenamiento")
display(datos_entrenamiento)

print("Datos de Prueba")
display(datos_prueba)

Datos de Entrenamiento


,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
0,310000,1,3,1,32,0,0,0,0,0,...,84373,57779,14163,8295,6000,4000,3000,1000,2000,0
1,10000,2,3,1,49,-1,-1,-2,-1,2,...,1690,1138,930,0,0,2828,0,182,0,1
2,50000,1,2,1,28,-1,-1,-1,0,-1,...,45975,1300,43987,0,46257,2200,1300,43987,1386,0
3,80000,2,3,1,52,2,2,3,3,3,...,40748,39816,40607,3700,1600,1600,0,1600,1600,1
4,270000,1,1,2,34,1,2,0,0,2,...,22448,15490,17343,0,4000,2000,0,2000,2000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20995,140000,2,2,1,27,2,-1,-1,-1,0,...,1580,804,728,752,800,1580,0,700,700,0
20996,130000,1,2,2,41,0,0,0,0,0,...,123107,42897,39378,4442,5200,5012,2500,5000,2000,0
20997,50000,1,3,2,23,0,0,0,0,0,...,28967,29829,30046,1973,1426,1001,1432,1062,997,0
20998,90000,2,3,2,25,0,0,0,0,0,...,5613,10113,10113,3000,3000,0,4500,0,3440,0


Datos de Prueba


,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
0,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
1,20000,1,3,2,35,-2,-2,-2,-2,-1,...,0,13007,13912,0,0,0,13007,1122,0,0
2,200000,2,3,2,34,0,0,2,0,0,...,2513,1828,3731,2306,12,50,300,3738,66,0
3,250000,1,1,2,29,0,0,0,0,0,...,59696,56875,55512,3000,3000,3000,3000,3000,3000,0
4,50000,2,3,3,23,1,2,0,0,0,...,28771,29531,30211,0,1500,1100,1200,1300,1100,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,20000,1,2,1,44,-2,-2,-2,-2,-2,...,2882,9235,1719,2890,2720,2890,9263,1824,1701,0
8996,360000,1,1,2,35,-1,-1,-2,-2,-2,...,0,0,0,0,0,0,0,0,0,0
8997,150000,1,1,2,35,-1,-1,-1,-1,-1,...,780,0,0,9054,0,783,0,0,0,0
8998,30000,1,2,2,37,4,3,2,-1,0,...,20878,20582,19357,0,0,22000,4200,2000,3100,1


## Paso 2: Divida los datasets en x_train, y_train, x_test, y_test

In [3]:
x_train = datos_entrenamiento.drop(columns=["default"])
y_train = datos_entrenamiento["default"]

x_test = datos_prueba.drop(columns=["default"])
y_test = datos_prueba["default"]

## Paso 3: Pipeline

In [4]:
#Transformación variables categóricas usando el método one-hot-encoding.
variables_categoricas = ["SEX", "EDUCATION", "MARRIAGE"]
transformer_variables_categoricas = OneHotEncoder()
preprocesador_1 = ColumnTransformer(
    transformers=[("transformer_1", transformer_variables_categoricas, variables_categoricas)],
    remainder="passthrough"
)

#Crear modelo de bosques aleatorios (rando forest).
modelo = RandomForestClassifier()

# Pipeline completo
pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocesador_1), 
        ("classifier",   modelo)
    ]
)

# Paso 4: Optimización de Hiperparámetros

In [5]:
# Optimice los hiperparametros del pipeline usando validación cruzada.
# Use 10 splits para la validación cruzada. Use la función de precision
# balanceada para medir la precisión del modelo.

In [6]:
# Establecer hiperparámetros a evaluar
param_grid = {
    "classifier__n_estimators"      : [50, 200],
    "classifier__max_depth"         : [None, 20],
   # "classifier__min_samples_split" : [2, 5, 10],
}

#Creación malla de hiperpárametros
busqueda_malla = GridSearchCV(
     estimator=pipeline, 
     param_grid=param_grid,
     scoring="balanced_accuracy",
     cv=5,
     verbose=2
)

#Entramiento de modelo
busqueda_malla.fit(x_train, y_train)




Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END classifier__max_depth=None, classifier__n_estimators=50; total time=   5.8s
[CV] END classifier__max_depth=None, classifier__n_estimators=50; total time=   5.4s
[CV] END classifier__max_depth=None, classifier__n_estimators=50; total time=   5.1s
[CV] END classifier__max_depth=None, classifier__n_estimators=50; total time=   4.9s
[CV] END classifier__max_depth=None, classifier__n_estimators=50; total time=   4.9s
[CV] END classifier__max_depth=None, classifier__n_estimators=200; total time=  20.4s
[CV] END classifier__max_depth=None, classifier__n_estimators=200; total time=  19.1s
[CV] END classifier__max_depth=None, classifier__n_estimators=200; total time=  25.7s
[CV] END classifier__max_depth=None, classifier__n_estimators=200; total time=  25.1s
[CV] END classifier__max_depth=None, classifier__n_estimators=200; total time=  26.3s
[CV] END classifier__max_depth=20, classifier__n_estimators=50; total time=   4.9s
[C

c:\Users\Olga\Documents\Predictiva\2024-2-LAB-01-prediccion-del-default-usando-rf-Paolabustos0510\.venv\Lib\site-packages\sklearn\compose\_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('transformer_1',
                                                                         OneHotEncoder(),
                                                                         ['SEX',
                                                                          'EDUCATION',
                                                                          'MARRIAGE'])])),
                                       ('classifier',
                                        RandomForestClassifier())]),
             param_grid={'classifier__max_depth': [None, 20],
                         'classifier__n_estimators': [50, 200]},
             scoring='balanced_accuracy', verbose=2)

In [7]:
#selecccionar el mejor modelo
mejor_modelo = busqueda_malla.best_estimator_
print(mejor_modelo)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('transformer_1',
                                                  OneHotEncoder(),
                                                  ['SEX', 'EDUCATION',
                                                   'MARRIAGE'])])),
                ('classifier', RandomForestClassifier(n_estimators=200))])


# Paso 5: Guardar Modelo

In [3]:
#joblib.dump(busqueda_malla, "../files/models/model.pkl")
with open("../files/models/model.pkl", "wb") as archivo:
    pickle.dump(busqueda_malla, archivo)

NameError: name 'busqueda_malla' is not defined

# Paso 6 y Paso 7: Cálculo Métricas y Matrices de Confusión

In [9]:
#Calcule las metricas de precision, precision balanceada, recall,
# y f1-score para los conjuntos de entrenamiento y prueba.
# Guardelas en el archivo files/output/metrics.json. Cada fila
# del archivo es un diccionario con las metricas de un modelo.
# Este diccionario tiene un campo para indicar si es el conjunto
# de entrenamiento o prueba.
# Guardelas en el archivo files/output/metrics.json. Cada fila
# del archivo es un diccionario con las metricas de un modelo.
# Este diccionario tiene un campo para indicar si es el conjunto
# de entrenamiento o prueba.
# Calcule las matrices de confusion para los conjuntos de entrenamiento y
# prueba. Guardelas en el archivo files/output/metrics.json. Cada fila
# del archivo es un diccionario con las metricas de un modelo.
# de entrenamiento o prueba

# Función para calcular métricas
def calcular_metricas(modelo, x, y, tipo_dataset): 
    y_pred = modelo.predict(x)
    diccionario_metricas = {
        "type" : "metrics",
        "dataset" : tipo_dataset,
        "precision" : precision_score(y, y_pred, zero_division=0),
        "balanced_accuracy" : balanced_accuracy_score(y, y_pred), 
        "recall" : recall_score(y, y_pred), 
        "f1" : f1_score(y, y_pred), 
    }
    return diccionario_metricas

 # Función para calcular matriz de confusión
def calcular_matriz_confusion(modelo, x, y, tipo_dataset):
    matriz_con = confusion_matrix(y, modelo.predict(x))
    diccionario_matriz = {
        "type": "cm_matrix",
        "dataset": tipo_dataset,
        "true_0": {"predicted_0": int(matriz_con[0, 0]), "predicted_1": int(matriz_con[0, 1])},
        "true_1": {"predicted_0": int(matriz_con[1, 0]), "predicted_1": int(matriz_con[1, 1])},
    }
    return diccionario_matriz

# Guardar métricas y matrices de confusión
valores = [
    calcular_metricas(mejor_modelo, x_train, y_train, "train"),
    calcular_metricas(mejor_modelo, x_test, y_test, "test"),
    calcular_matriz_confusion(mejor_modelo, x_train, y_train, "train"),
    calcular_matriz_confusion(mejor_modelo, x_test, y_test, "test"),
]
display(valores)




[{'type': 'metrics',
  'dataset': 'train',
  'precision': np.float64(0.9993645414107181),
  'balanced_accuracy': np.float64(0.9989558447751893),
  'recall': np.float64(0.9980960440025386),
  'f1': np.float64(0.9987298899237934)},
 {'type': 'metrics',
  'dataset': 'test',
  'precision': np.float64(0.6434563758389261),
  'balanced_accuracy': np.float64(0.67092295407772),
  'recall': np.float64(0.4017810371922472),
  'f1': np.float64(0.4946791357626572)},
 {'type': 'cm_matrix',
  'dataset': 'train',
  'true_0': {'predicted_0': 16270, 'predicted_1': 3},
  'true_1': {'predicted_0': 9, 'predicted_1': 4718}},
 {'type': 'cm_matrix',
  'dataset': 'test',
  'true_0': {'predicted_0': 6666, 'predicted_1': 425},
  'true_1': {'predicted_0': 1142, 'predicted_1': 767}}]

In [11]:
# Guardar archivo JSON
with open("../files/output/metrics.json", "w") as archivo:
    json.dump(valores, archivo)